# Exploration for corpus stats and initial topic modeling experiments

In [1]:
from pathlib import Path

import pandas as pd
import seaborn as sns


In [5]:
# Reusable functions 
def get_stop_list(stop_file):
    """Returns a set of stop words from file where
    they're one per line
    :param stop_file: Path object
    """
    text = stop_file.read_text(encoding="utf-8")
    stops = set([s.strip() for s in text.split("\n")])
    if '' in stops:
        stops.remove('')
    return stops


SyntaxError: invalid syntax (<ipython-input-5-eb9dc7d5e69e>, line 8)

In [3]:
CORPUS_DIR = Path('/home/virginia/workspace/topic-modeling-study/russian_novels')
DOCS_TSV = CORPUS_DIR / 'russian_novels.tsv'
STOP_LIST = CORPUS_DIR / 'russian_novels_stopped.txt'


    


In [4]:
stops = get_stop_list(STOP_LIST)

TypeError: unhashable type: 'list'